In [ ]:
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import random
import cv2
import datetime
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.chdir('/content/gdrive/My Drive/Data/')

In [ ]:
# !wget "https://github.com/OlafenwaMoses/ImageAI/releases/download/essentials-v5/resnet50_coco_best_v2.1.0.h5"

In [ ]:
# !pip install imageai --upgrade

In [ ]:
from imageai.Detection import ObjectDetection

execution_path = os.getcwd()

detector1 = ObjectDetection()
detector1.setModelTypeAsRetinaNet()
detector1.setModelPath( os.path.join(execution_path , "resnet50_coco_best_v2.1.0.h5"))
detector1.loadModel(detection_speed="flash")

In [ ]:
Attendance_Sheet = []

In [ ]:
date = datetime.datetime.now()
Date = date.strftime("%m-%d-%Y")
print(Date)
videos = [f for f in os.listdir('.') if f.endswith('.webm')]
print(videos)

05-25-2021
['20178091-17:1-5-25-2021.webm', '20178058-17:3-25-05-2021.webm']


In [ ]:
for video_name in videos:
  cap= cv2.VideoCapture("./"+video_name)
  i=0
  img = []
  while(cap.isOpened()):
      ret, frame = cap.read()
      if ret == False:
          break
      # cv2_imshow(frame)
      img.append(frame)
      i+=1
  cap.release()
  cv2.destroyAllWindows()

  no_frames = 20
  present = 0
  for x in range(no_frames):
    r = random.randrange(1,len(img)-1)
    detections = detector1.detectCustomObjectsFromImage(input_image=img[r], input_type="array", output_image_path=os.path.join(execution_path , "/output/image3new-custom.jpg"), minimum_percentage_probability=30)
    flag = 0
    for eachObject in detections:
        if(eachObject["name"] == 'person'):
          flag=1
    if(flag == 1):
      present = present+1
  present_p = present/no_frames
  if (present_p > 0.7):
    Attendance_Sheet.append({'Registration No.': video_name[0:8], 'Attendance': "Present", 'presence %': present_p*100})
  else:
    Attendance_Sheet.append({'Registration No.': video_name[0:8], 'Attendance': "Absent", 'presence %': present_p*100})
  os.remove("./"+video_name)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: 'detectCustomObjectsFromImage()' function has been deprecated and will be removed in future versions of ImageAI. 
 Kindly use 'detectObjectsFromImage()' 


In [ ]:
print(Attendance_Sheet)

[{'Registration No.': '20178091', 'Attendance': 'Present', 'presence %': 100.0}, {'Registration No.': '20178058', 'Attendance': 'Present', 'presence %': 100.0}]


In [ ]:
df1 = pd.DataFrame(Attendance_Sheet, columns=['Registration No.', 'Attendance', 'presence %'])
df1.to_excel("./Attendance_" +Date +".xlsx")